In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,111 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,839 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,153 kB]
Get:14

In [3]:
import pandas as pd
from pyspark.sql import functions as f

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .getOrCreate()
spark

In [5]:
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('contract_id', T.StringType(), False),
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('value', T.DoubleType(), True),
    T.StructField('value_source', T.StringType(), False),
    T.StructField('annotations', T.StringType(), True)
])

df_raw_time_series = spark.read.format('json') \
    .schema(data_schema) \
    .load('/content/drive/MyDrive/Colab Notebooks/project/raw_time_series/json')
df_raw_time_series = spark.createDataFrame(df_raw_time_series.rdd, schema=data_schema)
df_raw_time_series.printSchema()
df_raw_time_series.show()

root
 |-- contract_id: string (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- value: double (nullable = true)
 |-- value_source: string (nullable = false)
 |-- annotations: string (nullable = true)

+-------------------+-------------------+--------------------+------------+--------------------+
|        contract_id|          timestamp|               value|value_source|         annotations|
+-------------------+-------------------+--------------------+------------+--------------------+
| 04_02_111 _ CHR12 |2023-01-01 06:00:00| 0.02591860654732236| measurement|{"region":"Europe...|
| 04 _02_111 _CHR12 |2023-01-01 17:00:00| 0.07385444264936832| measurement|{"region":"Europe...|
| 04_02_111 _ CHR12 |2023-01-01 17:30:22| 0.08180149515221906| measurement|{"region":"Europe...|
| 04 _02_111 _CHR12 |2023-01-01 21:30:00| 0.08670661371854547| measurement|{"region":"Europe...|
|04 _ 02 _111_CHR12 |2023-01-02 00:30:00| 0.03597601881331959| measurement|{"region":"Europe...|
|04 

Data clean-up progress - if ruled more than once, all values will be turned to NULL

In [6]:
df_raw_time_series = (
    df_raw_time_series
    .withColumn('contract_id', f.regexp_replace(f.col('contract_id'), ' ', ''))
    .withColumn('value_source', f.when(f.col('value').isNull(), 'missing').otherwise(f.col('value_source')))
    .withColumn('timestamp',
                f.from_unixtime(f.round(f.unix_timestamp('timestamp') / (15 * 60)) * (15 * 60)))
    .withColumn('timestamp',f.date_format('timestamp', 'yyyy-MM-dd HH:mm'))
)
df_raw_time_series.show()

+---------------+----------------+--------------------+------------+--------------------+
|    contract_id|       timestamp|               value|value_source|         annotations|
+---------------+----------------+--------------------+------------+--------------------+
|04_02_111_CHR12|2023-01-01 06:00| 0.02591860654732236| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:00| 0.07385444264936832| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:30| 0.08180149515221906| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 21:30| 0.08670661371854547| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 00:30| 0.03597601881331959| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 05:30| 0.03638379308965683| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 10:45|            0.931575| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 18:30| 0.08816273670606922| measurement|{"region":"Europe...|
|04_02_111

Extracting a new `region` column from `annotations`

In [7]:
json_schema = T.StructType([
    T.StructField('region', T.StringType(), False)
])

df_raw_time_series = (
    df_raw_time_series
    .withColumn('region', f.from_json(f.col('annotations'), json_schema).getField('region'))
)
df_raw_time_series.show()

# check-up to see other regions
df_raw_time_series.select('region').distinct().show()

+---------------+----------------+--------------------+------------+--------------------+-------------+
|    contract_id|       timestamp|               value|value_source|         annotations|       region|
+---------------+----------------+--------------------+------------+--------------------+-------------+
|04_02_111_CHR12|2023-01-01 06:00| 0.02591860654732236| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:00| 0.07385444264936832| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:30| 0.08180149515221906| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 21:30| 0.08670661371854547| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 00:30| 0.03597601881331959| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 05:30| 0.03638379308965683| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-03 10:45|            0.931575| measurem

Customers with invalid regions will be removed from the database and saved on disk in a separate location.
- Suppose the regions respect the following format: `continent/city`

In [8]:
df_invalid_region = (
    df_raw_time_series
    .filter((f.col('region').isNull()) |
            (f.trim(f.col('region')) == '') |
            (f.regexp_extract(f.col('region'), r'^[A-Za-z]+(?:[\s-][A-Za-z]+)*\/[A-Za-z]+(?:[\s-][A-Za-z]+)*$', 0) == ''))
)

print("Count:", df_raw_time_series.count())

print("First variant - values after filtering directly:",
    df_raw_time_series
    .filter(~((f.col('region').isNull()) |
            (f.trim(f.col('region')) == '') |
            (f.regexp_extract(f.col('region'), r'^[A-Za-z]+(?:[\s-][A-Za-z]+)*\/[A-Za-z]+(?:[\s-][A-Za-z]+)*$', 0) == ''))).count()
)

print("Second variant - values after filtering via subtract:", df_raw_time_series.subtract(df_invalid_region).count())

df_raw_time_series = df_raw_time_series.subtract(df_invalid_region)
df_invalid_region.show()
df_invalid_region.write.json('/content/drive/MyDrive/Colab Notebooks/project/invalid_regions', mode="overwrite")

Count: 3549244
First variant - values after filtering directly: 3444112
Second variant - values after filtering via subtract: 3443374
+---------------+----------------+--------------------+------------+--------------------+--------+
|    contract_id|       timestamp|               value|value_source|         annotations|  region|
+---------------+----------------+--------------------+------------+--------------------+--------+
|01_02_155_CHR98|2023-01-01 00:15| 0.24727942049503326| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 01:00|                NULL|     missing|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 01:30| 0.09512702375650406| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 02:00| 0.26903796195983887| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 03:45| 0.10639405995607376| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 04:30|  0.1394430249929428| measurement|{"regi

Extract date from the `timestamp` column in a new `utc_date` column

In [9]:
df_raw_time_series = (
    df_raw_time_series
    .withColumn('utc_date',f.date_format('timestamp', 'yyyy-MM-dd'))
)
df_raw_time_series.show()

+---------------+----------------+--------------------+------------+--------------------+-------------+----------+
|    contract_id|       timestamp|               value|value_source|         annotations|       region|  utc_date|
+---------------+----------------+--------------------+------------+--------------------+-------------+----------+
|04_02_111_CHR01|2023-01-04 05:00| 0.05741286060061546| measurement|{"region":"Europe...|Europe/Berlin|2023-01-04|
|04_02_111_CHR01|2023-01-06 00:00|    0.11682207668514| measurement|{"region":"Europe...|Europe/Berlin|2023-01-06|
|04_02_111_CHR01|2023-01-24 16:00|0.051117820688296026| measurement|{"region":"Europe...|Europe/Berlin|2023-01-24|
|04_02_111_CHR01|2023-02-20 21:45|  0.2472069555955816| measurement|{"region":"Europe...|Europe/Berlin|2023-02-20|
|04_02_111_CHR01|2023-02-28 20:30|  0.2959699254694412| measurement|{"region":"Europe...|Europe/Berlin|2023-02-28|
|04_02_111_CHR01|2023-03-07 21:30|  0.2134278062811737| measurement|{"region":"E

Calculate local date for the date and time of `timestamp`, based on the region, in a new `local_timestamp` column

In [10]:
df_raw_time_series = df_raw_time_series.withColumn(
    "local_timestamp",
    f.from_utc_timestamp(df_raw_time_series["timestamp"], df_raw_time_series["region"])
)

df_raw_time_series.show()

+---------------+----------------+--------------------+------------+--------------------+-------------+----------+-------------------+
|    contract_id|       timestamp|               value|value_source|         annotations|       region|  utc_date|    local_timestamp|
+---------------+----------------+--------------------+------------+--------------------+-------------+----------+-------------------+
|04_02_111_CHR01|2023-01-04 05:00| 0.05741286060061546| measurement|{"region":"Europe...|Europe/Berlin|2023-01-04|2023-01-04 06:00:00|
|04_02_111_CHR01|2023-01-06 00:00|    0.11682207668514| measurement|{"region":"Europe...|Europe/Berlin|2023-01-06|2023-01-06 01:00:00|
|04_02_111_CHR01|2023-01-24 16:00|0.051117820688296026| measurement|{"region":"Europe...|Europe/Berlin|2023-01-24|2023-01-24 17:00:00|
|04_02_111_CHR01|2023-02-20 21:45|  0.2472069555955816| measurement|{"region":"Europe...|Europe/Berlin|2023-02-20|2023-02-20 22:45:00|
|04_02_111_CHR01|2023-02-28 20:30|  0.2959699254694412|

Extract date from the `local_timestamp` column in a new `local_date` column

In [11]:
df_raw_time_series = df_raw_time_series.withColumn(
    "local_date",
    f.to_date(df_raw_time_series["local_timestamp"])
)

df_raw_time_series.show()


+---------------+----------------+--------------------+------------+--------------------+-------------+----------+-------------------+----------+
|    contract_id|       timestamp|               value|value_source|         annotations|       region|  utc_date|    local_timestamp|local_date|
+---------------+----------------+--------------------+------------+--------------------+-------------+----------+-------------------+----------+
|04_02_111_CHR01|2023-01-04 05:00| 0.05741286060061546| measurement|{"region":"Europe...|Europe/Berlin|2023-01-04|2023-01-04 06:00:00|2023-01-04|
|04_02_111_CHR01|2023-01-06 00:00|    0.11682207668514| measurement|{"region":"Europe...|Europe/Berlin|2023-01-06|2023-01-06 01:00:00|2023-01-06|
|04_02_111_CHR01|2023-01-24 16:00|0.051117820688296026| measurement|{"region":"Europe...|Europe/Berlin|2023-01-24|2023-01-24 17:00:00|2023-01-24|
|04_02_111_CHR01|2023-02-20 21:45|  0.2472069555955816| measurement|{"region":"Europe...|Europe/Berlin|2023-02-20|2023-02-20

Extracting from the `annotations` column of electric vehicle consumption (EV), battery (BATTERY_IN) and consumption sent to the electric network (GRID_SELL) in the `sent_to_ev`, `sent_to_battery` and `sent_to_grid` columns. If the value is missing, consumption is 0.

Extraction from the `annotations` of the energy received from the solar panels (PV) and battery (BATTERY_OUT) in the `received_from_pv` and `received_from_battery` columns. If the value is missing, the energy received is 0.

In [41]:
# to show other features (and not only the region)
df_raw_time_series.select("annotations").show(300, truncate=False)

+--------------------------------------------------------------------------------------+
|annotations                                                                           |
+--------------------------------------------------------------------------------------+
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Berlin"}                                                            |
|{"region":"Europe/Be

In [12]:
# JSON schema for the events list - this includes every feature mentioned
json_schema = T.StructType([
    T.StructField("events", T.StructType([
        T.StructField("EV", T.StringType()),
        T.StructField("PV", T.StringType()),
        T.StructField("BATTERY_IN", T.StringType()),
        T.StructField("BATTERY_OUT", T.StringType()),
        T.StructField("GRID_SELL", T.StringType()),
    ]))
])

# extract the annotations in a column
df_raw_time_series = (
    df_raw_time_series.withColumn(
      "annotation_json",
      f.from_json(f.col("annotations"), json_schema)
))

# then get the values from the "events" part
df_raw_time_series = (
    df_raw_time_series
    .withColumn("sent_to_ev",
      f.coalesce(f.col("annotation_json.events.EV").cast(T.DoubleType()), f.lit(0)))
    .withColumn("received_from_pv",
      f.coalesce(f.col("annotation_json.events.PV").cast(T.DoubleType()), f.lit(0)))
    .withColumn("sent_to_battery",
      f.coalesce(f.col("annotation_json.events.BATTERY_IN").cast(T.DoubleType()), f.lit(0)))
    .withColumn("received_from_battery",
      f.coalesce(f.col("annotation_json.events.BATTERY_OUT").cast(T.DoubleType()), f.lit(0)))
    .withColumn("sent_to_grid",
      f.coalesce(f.col("annotation_json.events.GRID_SELL").cast(T.DoubleType()), f.lit(0)))
)

# drop the annotation column
df_raw_time_series = (
    df_raw_time_series
    .drop("annotation_json")
    .drop("annotations")
)

df_raw_time_series.show(300, truncate=False)


+---------------+----------------+--------------------+------------+-------------+----------+-------------------+----------+-----------------+----------------+-----------------+---------------------+-----------------+
|contract_id    |timestamp       |value               |value_source|region       |utc_date  |local_timestamp    |local_date|sent_to_ev       |received_from_pv|sent_to_battery  |received_from_battery|sent_to_grid     |
+---------------+----------------+--------------------+------------+-------------+----------+-------------------+----------+-----------------+----------------+-----------------+---------------------+-----------------+
|04_02_111_CHR01|2023-01-04 05:00|0.05741286060061546 |measurement |Europe/Berlin|2023-01-04|2023-01-04 06:00:00|2023-01-04|0.0              |0.0             |0.0              |0.0                  |0.0              |
|04_02_111_CHR01|2023-01-06 00:00|0.11682207668514    |measurement |Europe/Berlin|2023-01-06|2023-01-06 01:00:00|2023-01-06|0.0 